In [5]:
import pandas as pd
from datetime import datetime
import backtrader as bt
import matplotlib.pyplot as plt 
import tushare as ts
import akshare as ak
import sys,os


sys.stdout.encoding = 'utf_8_sig'
plt.rcParams["font.sans-serif"] = ["SimHei"]
plt.rcParams["axes.unicode_minus"] = False

In [13]:
a = ak.stock_financial_abstract_ths(symbol="300076",indicator="按报告期")
print(a.columns)
a.head()

Index(['报告期', '净利润', '净利润同比增长率', '扣非净利润', '扣非净利润同比增长率', '营业总收入', '营业总收入同比增长率',
       '基本每股收益', '每股净资产', '每股资本公积金', '每股未分配利润', '每股经营现金流', '销售净利率', '销售毛利率',
       '净资产收益率', '净资产收益率-摊薄', '营业周期', '存货周转率', '存货周转天数', '应收账款周转天数', '流动比率',
       '速动比率', '保守速动比率', '产权比率', '资产负债率'],
      dtype='object')


,报告期,净利润,净利润同比增长率,扣非净利润,扣非净利润同比增长率,营业总收入,营业总收入同比增长率,基本每股收益,每股净资产,每股资本公积金,...,净资产收益率-摊薄,营业周期,存货周转率,存货周转天数,应收账款周转天数,流动比率,速动比率,保守速动比率,产权比率,资产负债率
0,2007-12-31,2539.24万,False,2553.92万,False,1.60亿,False,0.7100,2.19,0.39,...,29.52%,291.55,2.64,136.11,155.44,1.79,1.47,1.47,1.75,49.67%
1,2008-12-31,3989.16万,57.10%,3226.80万,26.35%,1.76亿,10.19%,1.0100,3.18,0.50,...,31.88%,270.94,2.71,132.70,138.24,1.87,1.39,1.39,0.86,45.76%
2,2009-03-31,1320.11万,False,943.28万,False,3933.77万,False,0.3400,False,False,...,9.56%,False,False,False,False,False,False,False,False,False
3,2009-06-30,2349.12万,False,1934.14万,False,9208.99万,False,0.6000,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2009-09-30,2775.29万,False,False,False,1.30亿,False,0.3500,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
buy_date={}
sell_date={}
postion=set([])
stock_list=set([])
stock_columns={'总市值(万元)':[True,0.9],'市盈率(静态)':[False,1],'换手率(%)':[True,0.3]}
#stock_columns={'总市值(万元)':[True,0.8],'市盈率(静态)':[False,1]}
cw_columns={'营业收入环比增长率(%)(单季度)':[True,0.4]}
for i in range(0,len(time_id),30):
    stock_list_tem=stock_daily[(stock_daily["date"]==time_id[i+1])&(stock_daily["open"]!=stock_daily["close"])]["股票代码"].unique()
    
    stock_list_tem_0=stock_daily[(stock_daily["date"]==time_id[i])&(stock_daily["股票代码"].isin(stock_list_tem))]
    columns_list=list(stock_columns.keys())
    factor1_stock=stock_list_tem_0.sort_values(by=columns_list[0],ascending=stock_columns[columns_list[0]][0])["股票代码"].tolist()
    f_1_df=pd.Series(range(1,len(factor1_stock)+1),index=factor1_stock)*stock_columns[columns_list[0]][1]
    for j in columns_list[1:]:
        factor1_stock=stock_list_tem_0.sort_values(by=j,ascending=stock_columns[j][0])["股票代码"].tolist()
        f_1_df=f_1_df+pd.Series(range(1,len(factor1_stock)+1),index=factor1_stock)*stock_columns[j][1]
    
    cw_list_tem_0=cwzbsj[(cwzbsj["公告日期"]<time_id[i])&(cwzbsj["股票代码"].isin(stock_list_tem))].groupby("股票代码").last()
    if len(cw_list_tem_0)>0:
        for j in cw_columns.keys():
            factor1_stock=cw_list_tem_0.sort_values(by=j,ascending=cw_columns[j][0]).index.tolist()
            f_1_df=f_1_df+pd.Series(range(1,len(factor1_stock)+1),index=factor1_stock)*cw_columns[j][1]
    
    buy_list=f_1_df.sort_values().head(5).index.tolist()
    buy_date.update({str(time_id[i].date()):buy_list})
    sell_date.update({str(time_id[i].date()):list(postion-set(buy_list))})
    postion=set(buy_list)
    stock_list=stock_list|postion

In [6]:
#从csv文件读入所有股票k线数据
files = os.listdir('Data_k')
files.sort()
stock_list = []

stock_daily = pd.DataFrame()
for file in files[0:20]:
    df = pd.read_csv('Data_k/'+file,encoding='utf_8_sig')
    if len(df) == 0:
        print(file, "数据为空")
        continue
    stock_list.append(file.split('_')[1][:file.split('_')[1].find('c')-1])
    # 计算除权涨跌幅
    df['chg'] = df['close'].shift(1) / df['pre_close'] - 1
    df.iloc[0,11] = 0 #第11列为'chg'

    # =计算复权价: 计算所有因当当中用到的价格, 都使用复权价
    df['adj_factor'] = (1 + df['chg']).cumprod()

    label = 'after'  # 选择复权模式的标签
    # 计算后复权价格
    if label == 'after':
        df['adj_after_close'] = df['adj_factor'] * df['close']
    # 计算前复权价格
    if label == 'pre':
        df['adj_pre_close'] = df['adj_factor'] * (
                df.iloc[-1]['close'] / df.iloc[-1]['adj_factor'])
    stock_daily = pd.concat([stock_daily, df], axis=0)
    
stock_daily['trade_date'] = pd.to_datetime(stock_daily['trade_date'].astype("str"))
stock_daily.set_index("trade_date",inplace=True,drop=True)

C信凯科技_001335.SZ.csv 数据为空
C泰禾股份_301665.SZ.csv 数据为空
C泰鸿_603210.SH.csv 数据为空


In [19]:
stock_daily[stock_daily["ts_code"]=="300076.SZ"][["open","high","low","adj_after_close","vol"]]

,open,high,low,adj_after_close,vol
trade_date,,,,,
2010-04-30,75.00,76.90,68.21,68.260000,55444.68
2010-05-04,63.63,65.25,62.29,62.550000,20221.55
2010-05-05,60.68,64.58,60.65,64.450000,12354.31
2010-05-06,63.57,63.80,60.68,60.710000,13471.94
2010-05-07,59.40,61.35,59.00,60.200000,6862.39
...,...,...,...,...,...
2024-12-25,7.32,7.88,6.69,63.503012,510148.14
2024-12-26,7.60,7.88,7.52,63.004297,300890.67
2024-12-27,7.64,7.96,7.47,64.583561,237448.28


In [7]:
def get_data(syboml):
    stock = stock_daily[stock_daily["ts_code"]==syboml][["open","high","low","adj_after_close","vol"]]
    stock['openinterest'] = 0
    stock.rename(columns={"adj_after_close": "close", "vol": "volume"}, inplace=True)
    return stock

# 对齐导入的不同股票数据的日期，因为有些可能有停牌或者停市
n = len(stock_daily.index.unique())
d = pd.Series(range(0,n),index=stock_daily.index.unique()).sort_index()
#pd.concat([get_data("300076.SZ"),d],axis=1).ffill().bfill()

def get_data_1(syboml):
    stock = stock_daily[stock_daily["ts_code"]==syboml][["open","high","low","adj_after_close","vol"]]
    stock['openinterest'] = 0
    stock.rename(columns={"adj_after_close": "close", "vol": "volume"}, inplace=True)
    stock=pd.concat([stock,d],axis=1).fillna(method="bfill").fillna(method="ffill")
    return stock[["open","high","low","close","volume","openinterest"]]

In [4]:
for stock in datas['sec_code'].unique():
    # 日期对齐, 获取回测区间所有交易日
    data = pd.DataFrame(index=datas.index.unique())
    df = datas.query(f"sec_code=='{stock}'")[['open','high','low','close','volume','openinterest']] 
    data_ = pd.merge(data, df, left_index=True, right_index=True, how='left')
    # 缺失值处理
    data_.loc[:,['volume','openinterest']] = data_.loc[:,['volume','openinterest']].fillna(0)
    data_.loc[:,['open','high','low','close','EP','ROE']] = data_.loc[:,['open','high','low','close']].fillna(method='pad')
    data_.loc[:,['open','high','low','close','EP','ROE']] = data_.loc[:,['open','high','low','close']].fillna(0.0000001)

    # 导入数据
    datafeed = PandasData_more(dataname=data_,
                            fromdate = datetime.datetime(2019,1,31),
                            todate= datetime.datetime(2021,8,31),
                            timeframe = bt.TimeFrame.Months
                            )
    cerebro.adddata(datafeed,name=stock)
    #print(f"{stock} Done!")

# 获得收益率时序
ret = pd.Series(result[0].analyzers._TimeReturn.get_analysis())

NameError: name 'datas' is not defined

In [21]:
#策略接口
class my_strategy_date(bt.Strategy):
    #全局设定交易策略的参数

    def __init__(self):
        # 初始化交易指令、买卖价格和手续费
        self.order = None

    def next(self):
        # 检查是否持仓 
        # print(str(self.datetime.date(0)))
        if str(self.datetime.date(0)) in sell_date.keys(): 
            s_list=sell_date[str(self.datetime.date(0))] #取出对应日期的股票
            for i in s_list:
                self.order_target_percent(target=0,data=i)
                
        if str(self.datetime.date(0)) in buy_date.keys(): 
            s_list=buy_date[str(self.datetime.date(0))]
            for i in s_list:
                self.order_target_percent(target=0.9/len(s_list),data=i)
                

            
    def log(self, txt, dt=None):
        ''' 输出日志'''
        dt = dt or self.datas[0].datetime.date(0) # 拿现在的日期
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log(f"""买入{order.data._name}, 成交量{order.executed.size}，成交价{order.executed.price:.2f}""")
            elif order.issell():
                self.log(f"""卖出{order.data._name}, 成交量{order.executed.size}，成交价{order.executed.price:.2f}""")
            self.bar_executed = len(self)

        # Write down: no pending order
        self.order = None

In [ ]:
def backtest(start,end,stock_list,strategy):

    cerebro = bt.Cerebro()
    for i in stock_list:
        stock=get_data_1(i)
        data = bt.feeds.PandasData(dataname=stock,fromdate=start,todate=end)           
        cerebro.adddata(data,name=i) 
        
    cerebro.addstrategy(strategy) 
    cerebro.broker.setcash(100000) 
    cerebro.broker.setcommission(commission=0.002) 


    print(start.date(),end.date())
    print('初始资金: %.2f' % cerebro.broker.getvalue())
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name = 'SharpeRatio')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='DW')
    results = cerebro.run()
    strat = results[0]
    print('最终资金: %.2f' % cerebro.broker.getvalue())
    print('夏普比率:', strat.analyzers.SharpeRatio.get_analysis())
    print('回撤指标:', strat.analyzers.DW.get_analysis())
    
    return cerebro

In [37]:
start=datetime(2020,9,22)
end=datetime(2023,2,10)
buy_date={"2020-09-22":["300076.SZ","002748.SZ"],}
sell_date={"2023-02-02":["300076.SZ","002748.SZ"]}
backtest(start,end,stock_list[0:5],my_strategy_date)

2020-09-22 2023-02-10
初始资金: 100000.00
2020-09-23, 买入300076.SZ, 成交量925，成交价5.86
2020-09-23, 买入002748.SZ, 成交量2919，成交价7.36
2023-02-03, 卖出300076.SZ, 成交量-925，成交价5.75
2023-02-03, 卖出002748.SZ, 成交量-2919，成交价11.41
最终资金: 111589.14
夏普比率: OrderedDict({'sharperatio': 0.21561776210842395})
回撤指标: AutoOrderedDict({'len': 293, 'drawdown': 49.86342278778758, 'moneydown': 110981.18155310245, 'max': AutoOrderedDict({'len': 293, 'drawdown': 49.86342278778758, 'moneydown': 110981.18155310245})})
